# Лекция 2: Езикови модели и представяне на думи

## От Bag-of-Words към семантични вектори

**Продължителност:** 2-2.5 часа  
**Предпоставка:** Лекция 1 (ML основи, NLP мотивация)  
**Следваща лекция:** Токенизация

---
## 1. От Bag-of-Words към значение

### Какво научихме в Лекция 1?

В предишната лекция видяхме **Bag of Words (BoW)** и **TF-IDF** - прости но ефективни методи за представяне на текст.

### Ограничения на Bag-of-Words

❌ **Няма информация за ред на думите**
- "кучето гони котката" = "котката гони кучето"
- Различно значение, същото представяне!

❌ **Няма понятие за семантична прилика**
- "крал" и "кралица" са напълно различни в BoW
- Но семантично са много близки!

❌ **Разредени, високоразмерни представяния**
- Размерност = размер на речника (10,000+)
- Повечето стойности са 0

### Централен въпрос: Какво означава думите да имат "значение"?

Три подхода:

**1. Символни представяния (Symbolic)**
- Използват знания: WordNet, knowledge graphs
- Експертни системи с ръчно дефинирани отношения

**2. Дистрибутивна семантика (Distributional)**
- "Ще познаеш дума по компанията, която държи" (Firth, 1957)
- Базирани на съвместна срещаемост в текст

**3. Разпределени представяния (Distributed)**
- Плътни вектори (embeddings)
- Научени от данни (Word2Vec, GloVe)

### Пътна карта за тази лекция

```
WordNet → N-gram модели → Дистрибутивна семантика → Word2Vec
```

In [ ]:
# Импортиране на необходимите библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import itertools
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Настройка на визуализациите
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Всички библиотеки са заредени успешно!")

---
## 2. Символни представяния: WordNet

### Какво е WordNet?

**WordNet** е лексикална база данни на английския език, която организира думите в мрежа от семантични връзки.

- Създадена в Princeton University (1985-)
- Организирана в **synsets** (synonym sets) - множества от синоними
- Съдържа множество семантични отношения

### Ключови отношения в WordNet

**Синоними (Synonyms)**
- {car, automobile, motorcar, machine}
- Думи с подобно значение в synset

**Хипероними/Хипоними (Hypernyms/Hyponyms)**
- Йерархични отношения "е вид"
- animal → dog → beagle
- dog е хипоним на animal
- animal е хипероним на dog

**Мероними (Meronyms)**
- Отношения "част от"
- wheel е част от car
- engine е част от car

**Холоними (Holonyms)**
- Обратното на мероним
- car съдържа wheel

In [ ]:
# Опит за импортиране на nltk и WordNet
try:
    import nltk
    from nltk.corpus import wordnet as wn
    
    # Опит за изтегляне на WordNet данни ако не са налични
    try:
        wn.synsets('dog')
        print("✓ WordNet е наличен!")
    except:
        print("Изтегляне на WordNet данни...")
        nltk.download('wordnet', quiet=True)
        nltk.download('omw-1.4', quiet=True)
        print("✓ WordNet е наличен!")
        
    WORDNET_AVAILABLE = True
except ImportError:
    print("⚠️  NLTK не е инсталиран. Моля инсталирайте с: pip install nltk")
    WORDNET_AVAILABLE = False

In [ ]:
if WORDNET_AVAILABLE:
    print("🔍 Изследване на WordNet\n")
    print("="*70)
    
    # Synsets за думата "dog"
    word = "dog"
    synsets = wn.synsets(word)
    
    print(f"Synsets за '{word}':")
    for i, syn in enumerate(synsets[:3], 1):
        print(f"\n{i}. {syn.name()}")
        print(f"   Дефиниция: {syn.definition()}")
        print(f"   Примери: {syn.examples()}")
        print(f"   Леми: {[lemma.name() for lemma in syn.lemmas()]}")
    
    # Хипероними (по-общи понятия)
    dog_syn = wn.synset('dog.n.01')
    print("\n" + "="*70)
    print(f"\n🔼 Хипероними на 'dog' (по-общи понятия):")
    for hyper in dog_syn.hypernyms():
        print(f"  • {hyper.name()}: {hyper.definition()}")
    
    # Хипоними (по-специфични понятия)
    print(f"\n🔽 Хипоними на 'dog' (по-специфични понятия):")
    hyponyms = dog_syn.hyponyms()[:5]
    for hypo in hyponyms:
        print(f"  • {hypo.name()}: {hypo.definition()}")
    
    # Йерархия
    print("\n" + "="*70)
    print("\n🌳 Йерархия (от специфично към общо):")
    paths = dog_syn.hypernym_paths()
    if paths:
        path = paths[0]  # Вземаме първата пътека
        for syn in reversed(path[-5:]):  # Последните 5 нива
            print(f"  {'  ' * (len(path) - path.index(syn))}↓ {syn.name()}")
    
    # Семантична прилика
    print("\n" + "="*70)
    print("\n📏 Семантична прилика (path similarity):")
    
    cat = wn.synset('cat.n.01')
    car = wn.synset('car.n.01')
    
    dog_cat_sim = dog_syn.path_similarity(cat)
    dog_car_sim = dog_syn.path_similarity(car)
    
    print(f"  dog ↔ cat: {dog_cat_sim:.3f}")
    print(f"  dog ↔ car: {dog_car_sim:.3f}")
    print("\n  💡 По-високи стойности = по-голяма прилика")
else:
    print("WordNet демонстрацията е пропусната (NLTK не е наличен)")

### Ограничения на WordNet

**Проблеми:**

❌ **Интензивен човешки труд**
- Изисква експерти лингвисти
- Скъпо за създаване и поддръжка

❌ **Липсват нови думи и значения**
- "selfie", "cryptocurrency", "ChatGPT"
- Езикът еволюира по-бързо от WordNet

❌ **Субективни преценки**
- Кои думи са синоними?
- Колко близки са два концепта?

❌ **Няма нюанси в приликата**
- Дискретни отношения (има/няма)
- Не може да каже "крал е 0.8 подобен на кралица"

❌ **Не улавя контекст-зависимо значение**
- "банка" може да означава финансова институция или речен бряг
- WordNet има отделни synsets, но не знае кой да избере в контекст

**Въпреки това:** WordNet остава ценен ресурс за много NLP задачи!

---
## 3. Статистически езикови модели: N-грами

### Какво е езиков модел?

**Езиков модел (Language Model)** е вероятностна модел, която присвоява вероятност на последователности от думи.

**Формална дефиниция:**
$$P(w_1, w_2, ..., w_n)$$

### Защо се нуждаем от езикови модели?

**Приложения:**
- **Разпознаване на реч:** "recognize speech" срещу "wreck a nice beach"
- **Машинен превод:** Избор на най-естествения превод
- **Генериране на текст:** ChatGPT, автокомплийт
- **Правописна корекция:** "I have goed" → "I have gone"

### Вероятност на последователности

**Chain Rule (правило на веригата):**

$$P(w_1, w_2, ..., w_n) = \prod_{i=1}^{n} P(w_i | w_1, ..., w_{i-1})$$

**Проблем:** Твърде много възможни истории!

За речник с 50,000 думи:
- 1-дума история: 50,000 възможности
- 2-думи история: 2.5 милиарда
- 3-думи история: 125 трилиона!

### N-gram апроксимация

**Markov предположение:** Вероятността на дума зависи само от последните N-1 думи.

**Unigram (N=1):**
$$P(w_i)$$
- Игнорира контекст напълно

**Bigram (N=2):**
$$P(w_i | w_{i-1})$$
- Зависи само от предишната дума

**Trigram (N=3):**
$$P(w_i | w_{i-2}, w_{i-1})$$
- Зависи от последните 2 думи

### Оценка на вероятности

**Maximum Likelihood Estimation (MLE):**

Броим и нормализираме:

$$P(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i)}{C(w_{i-1})}$$

Където:
- $C(w_{i-1}, w_i)$ = брой на двойката $(w_{i-1}, w_i)$ в корпуса
- $C(w_{i-1})$ = брой на $w_{i-1}$ в корпуса

### Нека да построим Bigram модел!

In [ ]:
# Примерен корпус на български
corpus = [
    "котката гони мишката",
    "кучето гони котката",
    "котката спи на дивана",
    "кучето спи на килима",
    "мишката яде сирене",
    "котката яде риба",
    "кучето яде месо",
    "птицата лети високо",
    "котката гони птицата"
]

print("📚 Нашият корпус:")
for i, sent in enumerate(corpus, 1):
    print(f"{i}. {sent}")

In [ ]:
class BigramLanguageModel:
    def __init__(self):
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.vocab = set()
        
    def train(self, corpus):
        """Обучава модела на даден корпус"""
        for sentence in corpus:
            words = ['<START>'] + sentence.split() + ['<END>']
            
            for word in words:
                self.vocab.add(word)
            
            # Преброяваме биграми
            for i in range(len(words) - 1):
                w1, w2 = words[i], words[i+1]
                self.bigram_counts[w1][w2] += 1
                self.unigram_counts[w1] += 1
    
    def get_probability(self, w1, w2):
        """Връща P(w2 | w1)"""
        if self.unigram_counts[w1] == 0:
            return 0.0
        return self.bigram_counts[w1][w2] / self.unigram_counts[w1]
    
    def get_next_word_distribution(self, word):
        """Връща разпределението на следващите думи дадена текуща дума"""
        if word not in self.bigram_counts:
            return {}
        
        total = sum(self.bigram_counts[word].values())
        return {w: count/total for w, count in self.bigram_counts[word].items()}
    
    def generate_sentence(self, max_length=15):
        """Генерира изречение с модела"""
        sentence = ['<START>']
        
        for _ in range(max_length):
            current_word = sentence[-1]
            next_word_dist = self.get_next_word_distribution(current_word)
            
            if not next_word_dist or '<END>' in sentence:
                break
            
            # Избираме следваща дума базирано на вероятностите
            words = list(next_word_dist.keys())
            probs = list(next_word_dist.values())
            next_word = np.random.choice(words, p=probs)
            
            sentence.append(next_word)
            
            if next_word == '<END>':
                break
        
        return ' '.join(sentence[1:-1])  # Премахваме <START> и <END>

# Обучаваме модела
lm = BigramLanguageModel()
lm.train(corpus)

print("✓ Bigram моделът е обучен!")
print(f"\nРазмер на речника: {len(lm.vocab)} думи")
print(f"Брой биграми: {sum(len(counts) for counts in lm.bigram_counts.values())}")

In [ ]:
# Изследване на вероятности
print("🔍 Вероятности на биграми\n")
print("="*60)

test_bigrams = [
    ("котката", "гони"),
    ("котката", "спи"),
    ("котката", "лети"),  # Малко вероятно!
    ("кучето", "яде"),
]

for w1, w2 in test_bigrams:
    prob = lm.get_probability(w1, w2)
    print(f"P({w2} | {w1}) = {prob:.3f}")

# Разпределение на следващи думи
print("\n" + "="*60)
print("\n📊 Разпределение: Какво може да следва 'котката'?\n")

next_dist = lm.get_next_word_distribution('котката')
sorted_next = sorted(next_dist.items(), key=lambda x: x[1], reverse=True)

for word, prob in sorted_next:
    bar = '█' * int(prob * 50)
    print(f"{word:12s} {prob:.3f} {bar}")

In [ ]:
# Генериране на изречения
print("🎲 Генериране на изречения с Bigram модела\n")
print("="*60)

for i in range(5):
    sentence = lm.generate_sentence()
    print(f"{i+1}. {sentence}")

print("\n💡 Забележка: Изреченията следват статистическите модели от корпуса,")
print("   но може да не са напълно граматически правилни или смислени.")

### Предизвикателства на N-gram моделите

**1. Разреденост (Sparsity)**
- Много n-грами никога не се срещат в обучението
- $P(w_i | w_{i-1}) = 0$ за невиждани двойки
- Проблем: отхвърля валидни изречения

**2. Съхранение**
- Трябва да пазим броячи за всички n-грами
- За голям корпус: милиони биграми, милиарди триграми

**3. Липса на обобщение**
- "котката гони мишката" не помага за "кучето гони мишката"
- Всяка дума се третира като напълно различна

### Решение: Smoothing (изглаждане)

**Add-one (Laplace) smoothing:**

$$P(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i) + 1}{C(w_{i-1}) + V}$$

Където $V$ = размер на речника

**По-сложни методи:**
- Kneser-Ney smoothing
- Backoff и interpolation

📌 **Съвременно решение:** Невронни езикови модели (Лекция 4-5)!

---
## 4. Оценка на езикови модели

### Как измерваме качеството на езиков модел?

Искаме модел, който:
- Присвоява висока вероятност на валидни изречения
- Присвоява ниска вероятност на невалидни изречения

### Perplexity (объркване)

**Интуиция:** Колко "изненадан" е моделът от тестовите данни?

**Дефиниция:**

$$PP(W) = P(w_1, w_2, ..., w_N)^{-1/N}$$

Или еквивалентно:

$$PP = 2^{H}$$

Където $H$ е cross-entropy:

$$H = -\frac{1}{N}\sum_{i=1}^{N} \log_2 P(w_i | w_1, ..., w_{i-1})$$

### Cross-Entropy

**Cross-entropy** измерва средното количество информация (в битове), необходимо за кодиране на данните.

**Връзка с теорията на информацията:**
- По-добър модел → по-малко изненада → по-ниска cross-entropy
- Cross-entropy е минимална когато $P$ съвпада с истинското разпределение

### Интерпретация на Perplexity

**По-ниска perplexity = по-добър модел**

Perplexity може да се тълкува като **ефективен branching factor**:
- PP = 100 означава, че модела е "объркан" сякаш има 100 равновероятни избора при всяка стъпка
- PP = 10 е по-добре

**Типични стойности:**
- Bigram модел на Penn Treebank: ~150-200
- Trigram модел: ~100-150  
- Съвременни невронни LM: ~20-50
- Най-добрите LLM: ~5-15

In [ ]:
# Функция за изчисляване на perplexity
def compute_perplexity(model, test_corpus):
    """
    Изчислява perplexity на модел на тестов корпус
    """
    log_prob_sum = 0
    word_count = 0
    
    for sentence in test_corpus:
        words = ['<START>'] + sentence.split() + ['<END>']
        
        for i in range(len(words) - 1):
            w1, w2 = words[i], words[i+1]
            prob = model.get_probability(w1, w2)
            
            # Избягваме log(0)
            if prob > 0:
                log_prob_sum += np.log2(prob)
            else:
                # Penalize heavily for unseen bigrams
                log_prob_sum += np.log2(1e-10)
            
            word_count += 1
    
    # Cross-entropy
    cross_entropy = -log_prob_sum / word_count
    
    # Perplexity
    perplexity = 2 ** cross_entropy
    
    return perplexity, cross_entropy

# Тестов корпус (нови изречения)
test_corpus = [
    "котката гони мишката",
    "кучето спи на дивана",
    "птицата лети високо"
]

print("📊 Оценка на Bigram модела\n")
print("="*60)

# Perplexity на обучителния корпус
train_pp, train_ce = compute_perplexity(lm, corpus)
print(f"Тренировъчен набор:")
print(f"  Cross-entropy: {train_ce:.3f} bits")
print(f"  Perplexity: {train_pp:.3f}")

# Perplexity на тестовия корпус
test_pp, test_ce = compute_perplexity(lm, test_corpus)
print(f"\nТестов набор:")
print(f"  Cross-entropy: {test_ce:.3f} bits")
print(f"  Perplexity: {test_pp:.3f}")

print("\n💡 Интерпретация:")
print(f"   Моделът е 'объркан' сякаш има ~{test_pp:.0f} равновероятни избора")
print(f"   при всяка стъпка (за тестовия набор).")

### Защо Perplexity е важна?

**Perplexity е стандартна метрика за ВСИЧКИ езикови модели:**
- N-gram модели ✓
- RNN езикови модели ✓ (Лекция 4)
- Transformer модели ✓ (Лекция 5)
- GPT, BERT и съвременни LLM ✓ (Лекция 6-8)

**Връзка със следващите лекции:**
- Ще я използваме за да сравняваме различни модели
- Ще видим как невронните LM драстично подобряват perplexity
- Ще разберем scaling laws: как perplexity намалява с размера на модела

---
## 5. Дистрибутивна семантика

### Дистрибутивната хипотеза

> **"You shall know a word by the company it keeps"**  
> — J.R. Firth (1957)

**Централна идея:**
- Думи, които се появяват в подобни контексти, имат подобно значение
- "Кучето гони котката" vs "Кучето гони мишката"
- "котката" и "мишката" имат подобни контексти → подобно значение

### Co-occurrence матрици

**Две основни варианта:**

**1. Word-word матрица**
- Броим съседни думи в прозорец (напр. ±2 думи)
- Редове и колони = думи
- Стойност = колко пъти думите се срещат заедно

**2. Word-document матрица**
- Редове = думи
- Колони = документи
- Стойност = колко пъти дума се появява в документ
- Това е всъщност Bag-of-Words от Лекция 1!

### От броячи към вектори

Всяка дума се представя като:
- **Вектор** от context counts
- **Ред** в co-occurrence матрицата

### Измерване на прилика

**Cosine Similarity:**

$$\text{sim}(\mathbf{v}, \mathbf{w}) = \frac{\mathbf{v} \cdot \mathbf{w}}{||\mathbf{v}|| \cdot ||\mathbf{w}||} = \frac{\sum_{i} v_i w_i}{\sqrt{\sum_i v_i^2} \cdot \sqrt{\sum_i w_i^2}}$$

**Свойства:**
- Измерва ъгъла между вектори (не дължината)
- Стойност между -1 и 1
- 1 = идентични посоки
- 0 = ортогонални (независими)
- -1 = противоположни

### Нека да построим Co-occurrence матрица!

In [ ]:
# По-богат корпус за демонстрация
extended_corpus = [
    "котката гони мишката в градината",
    "кучето гони котката в парка",
    "котката спи на дивана вкъщи",
    "кучето спи на килима вкъщи",
    "мишката яде сирене в кухнята",
    "котката яде риба в кухнята",
    "кучето яде месо в градината",
    "птицата лети високо в небето",
    "котката гони птицата в парка",
    "птицата пее сутрин в дървото",
    "кучето лае силно навън",
    "котката мяука тихо вкъщи"
]

print("📚 Разширен корпус:")
for i, sent in enumerate(extended_corpus, 1):
    print(f"{i:2d}. {sent}")

In [ ]:
def build_cooccurrence_matrix(corpus, window_size=2):
    """
    Построява co-occurrence матрица от корпус
    
    Args:
        corpus: списък от изречения
        window_size: размер на контекстния прозорец
    
    Returns:
        DataFrame с co-occurrence матрица
    """
    # Събираме всички думи
    all_words = []
    for sent in corpus:
        all_words.extend(sent.split())
    
    # Речник и индекси
    vocab = sorted(set(all_words))
    word_to_idx = {word: i for i, word in enumerate(vocab)}
    
    # Инициализираме матрицата
    matrix = np.zeros((len(vocab), len(vocab)))
    
    # Попълваме матрицата
    for sent in corpus:
        words = sent.split()
        for i, target_word in enumerate(words):
            target_idx = word_to_idx[target_word]
            
            # Гледаме в прозореца около думата
            start = max(0, i - window_size)
            end = min(len(words), i + window_size + 1)
            
            for j in range(start, end):
                if i != j:  # Не броим думата със себе си
                    context_word = words[j]
                    context_idx = word_to_idx[context_word]
                    matrix[target_idx, context_idx] += 1
    
    return pd.DataFrame(matrix, index=vocab, columns=vocab)

# Построяваме матрицата
cooccur_matrix = build_cooccurrence_matrix(extended_corpus, window_size=2)

print("\n📊 Co-occurrence матрица (частична)\n")
print("="*70)

# Показваме интересни думи
interesting_words = ['котката', 'кучето', 'мишката', 'птицата', 'гони', 'яде', 'спи']
subset = cooccur_matrix.loc[interesting_words, interesting_words]
print(subset.to_string())

print(f"\nПълна матрица: {cooccur_matrix.shape[0]} x {cooccur_matrix.shape[1]}")
print(f"Брой ненулеви елементи: {np.count_nonzero(cooccur_matrix.values)}")
sparsity = 1 - np.count_nonzero(cooccur_matrix.values) / cooccur_matrix.size
print(f"Разреденост: {sparsity:.1%}")

In [ ]:
# Визуализация на матрицата
plt.figure(figsize=(10, 8))
sns.heatmap(subset, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Co-occurrence count'},
            square=True)
plt.title('Co-occurrence матрица (избрани думи)', fontsize=14, pad=15)
plt.xlabel('Контекстна дума', fontsize=12)
plt.ylabel('Целева дума', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n💡 Интерпретация:")
print("   По-тъмните клетки показват по-често срещани съвместни появи.")
print("   Забележете, че 'котката', 'кучето' и 'мишката' имат подобни модели!")

In [ ]:
# Намиране на най-сходни думи с cosine similarity
def find_most_similar(word, matrix, top_n=5):
    """
    Намира най-сходните думи на дадена дума
    """
    if word not in matrix.index:
        return []
    
    # Вектор на думата
    word_vector = matrix.loc[word].values.reshape(1, -1)
    
    # Изчисляваме cosine similarity с всички думи
    similarities = cosine_similarity(word_vector, matrix.values)[0]
    
    # Създаваме списък с (дума, сходство)
    word_similarities = list(zip(matrix.index, similarities))
    
    # Сортираме и вземаме топ N (без самата дума)
    word_similarities = [(w, s) for w, s in word_similarities if w != word]
    word_similarities.sort(key=lambda x: x[1], reverse=True)
    
    return word_similarities[:top_n]

print("🔍 Най-сходни думи (cosine similarity)\n")
print("="*70)

test_words = ['котката', 'кучето', 'гони', 'вкъщи']

for word in test_words:
    similar = find_most_similar(word, cooccur_matrix, top_n=5)
    print(f"\n'{word}' е най-сходна с:")
    for similar_word, similarity in similar:
        bar = '█' * int(similarity * 30)
        print(f"  {similar_word:12s} {similarity:.3f} {bar}")

### Проблеми с Co-occurrence матрици

**1. Много високоразмерни**
- Размерност = размер на речника (10,000+ думи)
- Матрица 10,000 x 10,000 = 100 милиона елемента

**2. Разредени (Sparse)**
- Повечето елементи са 0
- Неефективно за съхранение и изчисления

**3. Нямат ясно значение**
- Какво представляват измеренията?
- Трудно за интерпретиране

**4. Изчислително скъпи**
- Строене на матрицата изисква сканиране на целия корпус
- Изчисляване на прилики изисква операции с високоразмерни вектори

**Решение:** Dense embeddings (Word2Vec) → следваща секция!

---
## 6. Word Embeddings: Word2Vec

### Dense vs Sparse представяния

**Sparse (разредени):**
- One-hot vectors: [0, 0, 0, 1, 0, ..., 0]
- Co-occurrence vectors: [0, 3, 0, 0, 1, 0, ..., 5]
- Размерност = размер на речника (10,000+)
- Повечето стойности са 0

**Dense (плътни):**
- Continuous vectors: [0.23, -0.45, 0.67, ..., 0.12]
- Размерност: 50-300 (много по-малка!)
- Всички стойности са ненулеви

**Предимства на dense:**
- ✅ **Ефективност:** По-малко параметри, по-бързи изчисления
- ✅ **Обобщение:** Сходни думи имат сходни вектори
- ✅ **Научени от данни:** Автоматично откриват важни характеристики

### Word2Vec: Мотивация

**Централна идея:**
- Научи embeddings, които улавят семантични връзки
- Използвай контекста за да предвидиш думата (или обратно)
- **"You shall know a word by the company it keeps"** - но научено с невронна мрежа!

**Предложен от:** Томаш Миколов и екип в Google (2013)

### Две архитектури

#### 1. Skip-gram

**Задача:** Предвиди контекстни думи от централна дума

```
Вход:  [централна дума]
Изход: [контекстни думи]
```

**Пример:**
- Изречение: "котката гони мишката в градината"
- Централна дума: "мишката"
- Контекст (window=2): ["гони", "в", "градината"]

**Предимства:**
- Добър за редки думи
- По-добре улавя редки връзки

#### 2. CBOW (Continuous Bag-of-Words)

**Задача:** Предвиди централна дума от контекст

```
Вход:  [контекстни думи]
Изход: [централна дума]
```

**Предимства:**
- По-бърз за обучение
- По-добър за чести думи

### Обучителна цел

**Skip-gram objective:**

$$J(\theta) = \frac{1}{T}\sum_{t=1}^{T}\sum_{-c \leq j \leq c, j \neq 0} \log P(w_{t+j} | w_t)$$

Където:
- $T$ = брой думи в корпуса
- $c$ = размер на контекстния прозорец
- $w_t$ = централна дума
- $w_{t+j}$ = контекстна дума

**Вероятността се моделира с softmax:**

$$P(w_O | w_I) = \frac{\exp(\mathbf{v}_{w_O}^\top \mathbf{v}_{w_I})}{\sum_{w=1}^{V} \exp(\mathbf{v}_w^\top \mathbf{v}_{w_I})}$$

**Проблем:** Softmax е скъп за изчисляване (сума по целия речник!)

**Решение:** Negative sampling - вместо да изчисляваме вероятност за всички думи, сравняваме само с няколко негативни примера.

### Какво научава Word2Vec?

**1. Семантична прилика**
- `king` ≈ `queen`
- `dog` ≈ `puppy`
- `car` ≈ `automobile`

**2. Синтактични модели**
- `walking` ≈ `running` ≈ `swimming`
- Глаголи в едно и също време

**3. Аналогии**
- `king - man + woman ≈ queen`
- `Paris - France + Italy ≈ Rome`
- `walking - walk + swim ≈ swimming`

**Векторна аритметика работи!**

### Демонстрация с gensim (ако е наличен)

In [ ]:
# Проверка дали gensim е наличен
try:
    from gensim.models import Word2Vec
    from gensim.models.word2vec import LineSentence
    GENSIM_AVAILABLE = True
    print("✓ Gensim е наличен!")
except ImportError:
    GENSIM_AVAILABLE = False
    print("⚠️  Gensim не е инсталиран. Моля инсталирайте с: pip install gensim")
    print("   За демонстрация ще покажем концептуална визуализация.")

In [ ]:
if GENSIM_AVAILABLE:
    # Подготовка на корпус за Word2Vec
    # Word2Vec очаква списък от списъци с думи
    w2v_corpus = [sent.split() for sent in extended_corpus]
    
    print("🏋️ Обучаване на Word2Vec модел...\n")
    
    # Обучаване на модел
    # vector_size = размерност на embeddings (намалена за малкия корпус)
    # window = размер на контекстния прозорец
    # min_count = минимален брой срещания на дума
    # sg = 1 за skip-gram, 0 за CBOW
    model = Word2Vec(
        sentences=w2v_corpus,
        vector_size=50,
        window=3,
        min_count=1,
        sg=1,  # Skip-gram
        epochs=100,
        seed=42
    )
    
    print("✓ Моделът е обучен!")
    print(f"\nРечник: {len(model.wv)} думи")
    print(f"Размерност на embeddings: {model.wv.vector_size}")
    
    # Най-сходни думи
    print("\n" + "="*70)
    print("\n🔍 Най-сходни думи:\n")
    
    test_words = ['котката', 'кучето', 'гони', 'спи']
    
    for word in test_words:
        if word in model.wv:
            similar = model.wv.most_similar(word, topn=3)
            print(f"'{word}':")
            for similar_word, similarity in similar:
                print(f"  • {similar_word}: {similarity:.3f}")
            print()
    
    # Аналогии (може да не работят добре на малък корпус)
    print("="*70)
    print("\n🧮 Опит за аналогии (може да не работи перфектно на малък корпус):\n")
    
    try:
        # котката гони X, кучето гони ?
        result = model.wv.most_similar(
            positive=['кучето', 'мишката'],
            negative=['котката'],
            topn=1
        )
        print(f"котката:мишката :: кучето:?")
        print(f"  → {result[0][0]} (similarity: {result[0][1]:.3f})")
    except:
        print("  Недостатъчно данни за аналогии на този малък корпус")
    
    print("\n💡 Забележка: За по-добри резултати се нуждаем от по-голям корпус!")
    print("   Типично Word2Vec се обучава на милиони изречения.")
else:
    print("\n⚠️  Демонстрацията на Word2Vec е пропусната (gensim не е наличен)")
    print("\nЗа да експериментирате с Word2Vec:")
    print("  1. Инсталирайте gensim: pip install gensim")
    print("  2. Изпълнете отново тази клетка")

### Концептуална визуализация на embeddings

In [ ]:
# Концептуална визуализация (дори без gensim)
# Симулираме 2D проекция на word embeddings

np.random.seed(42)

# Групи думи
animals = ['котката', 'кучето', 'мишката', 'птицата']
actions = ['гони', 'спи', 'яде', 'лети', 'лае', 'мяука', 'пее']
places = ['градината', 'парка', 'вкъщи', 'кухнята', 'дивана', 'килима', 'дървото']

# Симулираме позиции (в реалността идват от t-SNE на embeddings)
# Животни са близо едни до други
animal_positions = np.random.randn(len(animals), 2) * 0.3 + np.array([0, 2])

# Действия са в друг клъстер
action_positions = np.random.randn(len(actions), 2) * 0.3 + np.array([2, 0])

# Места са в трети клъстер
place_positions = np.random.randn(len(places), 2) * 0.3 + np.array([-1.5, -1])

# Визуализация
plt.figure(figsize=(12, 8))

# Животни
plt.scatter(animal_positions[:, 0], animal_positions[:, 1], 
           s=200, c='red', alpha=0.6, marker='o', label='Животни')
for i, word in enumerate(animals):
    plt.annotate(word, (animal_positions[i, 0], animal_positions[i, 1]),
                fontsize=11, ha='center', va='center', fontweight='bold')

# Действия
plt.scatter(action_positions[:, 0], action_positions[:, 1], 
           s=200, c='blue', alpha=0.6, marker='s', label='Действия')
for i, word in enumerate(actions):
    plt.annotate(word, (action_positions[i, 0], action_positions[i, 1]),
                fontsize=11, ha='center', va='center', fontweight='bold')

# Места
plt.scatter(place_positions[:, 0], place_positions[:, 1], 
           s=200, c='green', alpha=0.6, marker='^', label='Места')
for i, word in enumerate(places):
    plt.annotate(word, (place_positions[i, 0], place_positions[i, 1]),
                fontsize=11, ha='center', va='center', fontweight='bold')

plt.xlabel('Измерение 1', fontsize=12)
plt.ylabel('Измерение 2', fontsize=12)
plt.title('Концептуална визуализация на Word Embeddings\n(2D проекция на векторно пространство)', 
         fontsize=14, pad=15)
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5, alpha=0.3)
plt.axvline(x=0, color='k', linewidth=0.5, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Интерпретация:")
print("   • Семантично сходни думи се групират заедно")
print("   • Животните формират клъстер")
print("   • Действията формират друг клъстер")
print("   • Местата формират трети клъстер")
print("\n   В реалните embeddings тези клъстери се появяват автоматично!")

### Ограничения и пристрастия

**1. Един вектор на дума (няма полисемия)**
- "банка" има едно представяне
- Но може да означава финансова институция или речен бряг
- Word2Vec усреднява всички значения

**2. Улавя пристрастия в данните**
- Ако корпусът съдържа стереотипи, embeddings ще ги улавят
- Известни примери:
  - `doctor - man + woman ≈ nurse`
  - `programmer - man + woman ≈ homemaker`
- Етични въпроси за NLP системи!

**3. Статични (не се променят с контекста)**
- Веднъж обучени, embeddings са фиксирани
- Не се адаптират към различни значения в различни изречения

**Решение за 1 и 3:** Contextual embeddings (ELMo, BERT) → **Лекция 5**!

---
## 7. Практически съображения

### Използване на pre-trained embeddings

**Популярни pre-trained модели:**

**1. Google News Word2Vec**
- 3 милиона думи
- 300 измерения
- Обучен на Google News корпус

**2. GloVe (Global Vectors)**
- Stanford NLP
- Различни версии: Wikipedia, Twitter, Common Crawl
- 50d, 100d, 200d, 300d

**3. FastText**
- Facebook AI
- Базиран на поддуми (subwords)
- Може да работи с Out-Of-Vocabulary (OOV) думи!

**Кога да използваме pre-trained?**
- ✅ Малки datasets (transfer learning)
- ✅ Общи текстове (новини, уеб)
- ✅ Ограничени изчислителни ресурси

### Обучаване на собствени embeddings

**Кога да обучим свои?**
- ✅ Специфичен домейн (медицина, право, финанси)
- ✅ Специализиран речник
- ✅ Достатъчно данни (милиони думи)

**Хиперпараметри:**
- `vector_size`: Размерност (50-300)
- `window`: Размер на контекстния прозорец (5-10)
- `min_count`: Минимална честота на дума (1-5)
- `sg`: Skip-gram (1) или CBOW (0)
- `epochs`: Брой епохи (5-20)

### Избор на представяне

**Bag-of-Words / TF-IDF**
- ✅ Прости baseline модели
- ✅ Интерпретируеми
- ✅ Работят за малки datasets
- ❌ Не улавят семантика

**Word2Vec / GloVe**
- ✅ Улавят семантични връзки
- ✅ Dense представяния
- ✅ Transfer learning
- ❌ Статични (няма контекст)
- ❌ Проблем с полисемия

**Contextual Embeddings (BERT, ELMo)**
- ✅ Различни embeddings за различни контексти
- ✅ State-of-the-art performance
- ❌ По-сложни и бавни
- ❌ Изискват повече ресурси

📌 **Ще видим contextual embeddings в Лекция 5!**

### Как да заредим pre-trained embeddings

In [ ]:
print("📦 Зареждане на pre-trained embeddings\n")
print("="*70)

if GENSIM_AVAILABLE:
    print("Пример за зареждане на Google News Word2Vec:")
    print("""\n# Изтегляне (еднократно):
# wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

# Зареждане:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', 
    binary=True
)

# Използване:
similar = model.most_similar('king', topn=5)
analogy = model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
""")
    
    print("\nПример за зареждане на GloVe:")
    print("""\n# Изтегляне от: https://nlp.stanford.edu/projects/glove/

# Зареждане:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = 'glove.6B.100d.txt'
word2vec_file = 'glove.6B.100d.word2vec.txt'

# Конвертиране на формат
glove2word2vec(glove_file, word2vec_file)

# Зареждане
model = KeyedVectors.load_word2vec_format(word2vec_file)
""")
else:
    print("⚠️  За зареждане на pre-trained embeddings инсталирайте gensim:")
    print("   pip install gensim")

print("\n💡 Съвет: Pre-trained embeddings са отличен начин да започнете!")
print("   Винаги пробвайте готови embeddings преди да обучавате свои.")

---
## 8. Обобщение и мост към Лекция 3

### 🎓 Ключови изводи

#### Днес научихме:

**1. Три подхода за представяне на значение**
- **Символни:** WordNet - експертни знания, но скъпи и непълни
- **Дистрибутивни:** Co-occurrence матрици - статистически, но разредени
- **Разпределени:** Word2Vec - dense, научени, улавят семантика

**2. Езикови модели**
- **N-gram модели:** Предсказват думи базирано на история
- **Perplexity:** Стандартна метрика за оценка
- Основа за разбиране на съвременни LLM

**3. Word2Vec**
- Skip-gram vs CBOW
- Улавя семантични и синтактични модели
- Векторна аритметика: `king - man + woman ≈ queen`
- Ограничения: един вектор на дума, статични

**4. Практически съображения**
- Pre-trained embeddings за transfer learning
- Избор на представяне зависи от задачата

### 🔜 Следваща лекция: Токенизация

**Критични въпроси, които ще отговорим:**

❓ Какво точно е "дума"?  
❓ Как обработваме "New York" или "don't"?  
❓ Как работим с Out-Of-Vocabulary думи?  
❓ Какво е BPE (Byte Pair Encoding)?  
❓ Как токенизират GPT, BERT и съвременните LLM?  

**Защо е важно:**
- Токенизацията е КРИТИЧНА за всички съвременни LLM
- ChatGPT, Claude използват subword токенизация
- Правилната токенизация подобрява performance драстично

### 📚 Препоръчително четене

**Основни статии:**

1. **"Efficient Estimation of Word Representations in Vector Space"** - Mikolov et al. (2013)
   - Оригиналната Word2Vec статия

2. **"Distributed Representations of Words and Phrases"** - Mikolov et al. (2013)
   - Negative sampling и phrase embeddings

3. **"GloVe: Global Vectors for Word Representation"** - Pennington et al. (2014)
   - Алтернативен метод за word embeddings

**Учебници:**

1. **"Speech and Language Processing"** - Jurafsky & Martin (3rd ed.)
   - Chapter 3: N-gram Language Models
   - Chapter 6: Vector Semantics and Embeddings

2. **Stanford CS224N Lecture 2:** Word Vectors

### 🏋️ Упражнения за вкъщи

**1. N-gram експерименти**
- Обучете trigram модел на корпус
- Сравнете perplexity на unigram, bigram, trigram
- Генерирайте текст с различни модели

**2. Word2Vec изследване**
- Заредете pre-trained Word2Vec (Google News)
- Намерете най-сходни думи за различни concepts
- Тествайте аналогии: `man:woman :: king:?`
- Анализирайте какви връзки улавя

**3. Сравнение на представяния**
- Имплементирайте sentiment classifier с:
  - Bag-of-words
  - TF-IDF
  - Averaged Word2Vec embeddings
- Сравнете performance и interpretability

**4. Визуализация**
- Използвайте t-SNE за визуализация на Word2Vec
- Идентифицирайте клъстери от сходни думи
- Анализирайте какви semantic regions откриваме

---
## Край на Лекция 2

### Благодаря за вниманието! 🎉

**Въпроси? 🤔**

---

**Следваща лекция:** Токенизация и subword models  
**Дата:** [TBD]